In [7]:
import json
import os
import time
from datetime import date, datetime, timedelta
from pathlib import Path

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from Mods import pandas_mod as pdm
from Mods import date_mod as dtm

### 自訂函式

In [8]:
def read_or_build(folder, file, columns):
    """檢查路徑檔案是否存在，若有則讀取，無則建立空表格"""
    file_path = os.path.join(folder, file)
    path = Path(file_path)

    # 若檔案不存在則先新建空的df並存檔
    if path.exists():
        df = pd.read_csv(file_path)
    else:
        columns = get_col()
        df = pd.DataFrame(columns=columns)

    return df


def get_col():
    """取得欄位名list"""
    return [
        'flight_NO',
        'flight_type',
        'flight_company',
        'fly_distance',
        'departure_airport_code',
        'departure_city',
        'arrival_airport_code',
        'arrival_city',
        'departure_date',
        'leave_gate_estimate',
        'leave_gate_actual',
        'departure_time_estimate',
        'departure_time_actual',
        'departure_timezone',
        'arrival_date',
        'landing_time_estimate',
        'landing_time_actual',
        'arrive_gate_estimate',
        'arrive_gate_actual',
        'arrive_timezone',
        'link'
    ]


def source_list_mask(df_list, df_table):
    """根據規則篩選來源資料（暫廢）"""
    today = pd.Timestamp.now()

    mask_1 = (df_list['sync'] == 0)
    mask_2 = ((today - df_list['query_date']).dt.days >= 2)
    mask_3 = df_list['link'].isin(df_table['link'])

    source = df_list[mask_1 & mask_2 & ~mask_3]

    return source


def get_page_source(url, driver):
    """透過selenium取得網頁html內容"""
    driver.get(url)

    # 網頁內有JavaScript動態生成內容，故設定等待網頁讀取完畢後再動作
    wait = WebDriverWait(driver, 10)
    element = wait.until(
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, "flightPageSummaryDepartureDay"))
    )

    page_source = driver.page_source

    return page_source


def trans_date_from_chinese(chinese_date):
    """將中文日期格式轉為datetime格式"""
    clean_date = chinese_date.split("(")[0].strip()
    fmt = "%Y年 %m月 %d日"
    trans = datetime.strptime(clean_date, fmt)

    return trans


def find_tag(div_list, target_str: str):
    """用於尋找特定字串標籤的index"""
    target = 0
    for i in div_list:
        if i.get_text() == target_str:
            break
        else:
            target += 1
    return target


def safe_extract(func):
    """判斷一個soup物件是否存在/有值，若沒有則回傳None"""
    try:
        return func()
    except (IndexError, AttributeError):
        return None


def gate_exist(soup):
    """判斷一個航班頁面中是否有到/離閘口資料"""
    gate = 0
    keywords = ["閘口", "停机位", "Gate", "gate"]
    x = soup
    for i in x('div'):
        if any(keyword in i.text for keyword in keywords):
            gate = 1
            break
    return gate


def split_tz(time_str: str):
    """將帶有時區的時間字串分割，得到[time, timezone]列表"""
    if "(" in time_str:
        time_str = time_str.split("(")[0].strip()

    time_, tz = time_str.split(' ')
    return time_, tz


def crawl_flight_data(soup, url, gate_exist: bool):
    """當該班機有到/離閘門資料時使用的爬蟲"""
    flight_data = []

    # 班機基本資料。較容易在各網頁中出現差異，故先使用函式取得定位，再去取得資訊
    div_list = soup('div', class_='flightPageDataLabel')

    # 航班編號、機型、航空公司
    flight_data.append(safe_extract(lambda: soup(
        'div', class_='flightPageIdent')[0].h1.text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[
                       0]('div', class_='flightPageData')[0].text.strip().replace('\xa0', ' ')))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[
                       2]('div', class_='flightPageData')[0].text.strip().split('\n')[0]))

    # 飛行距離
    for n in range(0, len(soup('div', class_='flightPageDataLabel'))):
        if soup('div', class_='flightPageDataLabel')[n].text == "距離":
            num = n

    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageDataRow')[
                       num].span.text.strip().replace(',', '').replace("\n", "").replace("\t", "")))

    # 起飛機場、起飛城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[
                       0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryOrigin')[
                       0]('span', class_='flightPageSummaryCity')[0].text.strip()))

    # 降落機場、降落城市
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[
                       0]('span', class_='displayFlexElementContainer')[0].text.strip()))
    flight_data.append(safe_extract(lambda: soup('div', class_='flightPageSummaryDestination')[
                       0]('span', class_='destinationCity')[0].text.strip()))

    # 起飛日期
    flight_data.append(safe_extract(lambda: soup(
        'span', class_='flightPageSummaryDepartureDay')[0].text))

    if gate_exist == 1:
        # 預計/實際離開閘門時間
        lg_e_time, lg_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                      0].span.text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        lg_a_time, lg_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[0](
            'div', class_="flightPageDataActualTimeText")[0].text.strip().replace('\xa0', ' ').replace('\\n', '').replace('\\t', '')))

        flight_data.append(lg_e_time)
        flight_data.append(lg_a_time)

        # 預計/實際起飛時間
        d_e_time, d_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    1]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        d_a_time, d_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    1]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

        flight_data.append(d_e_time)
        flight_data.append(d_a_time)

    else:
        # 預計/實際離開閘門時間
        flight_data.append(None)
        flight_data.append(None)

        # 預計/實際起飛時間
        d_e_time, d_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    0]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        d_a_time, d_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    0]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

        flight_data.append(d_e_time)
        flight_data.append(d_a_time)

    # 起飛時區
    flight_data.append(d_a_tz)

    # 抵達日期
    flight_data.append(safe_extract(lambda: soup(
        'span', class_='flightPageSummaryArrivalDay')[0].text))

    if gate_exist == 1:
        # 預計/實際降落時間
        a_e_time, a_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    2]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        a_a_time, a_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    2]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

        flight_data.append(a_e_time)
        flight_data.append(a_a_time)

        # 預計/實際抵達閘門時間
        ag_e_time, ag_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                      3]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        ag_a_time, ag_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[3](
            'div', class_='flightPageDataActualTimeText')[0].span.text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

        flight_data.append(ag_e_time)
        flight_data.append(ag_a_time)

    else:
        # 預計/實際降落時間
        a_e_time, a_e_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    1]('span')[1].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))
        a_a_time, a_a_tz = split_tz(safe_extract(lambda: soup('div', class_='flightPageDataTimesChild')[
                                    1]('span')[0].text.strip().replace('\xa0', ' ').replace('\n', '').replace('\t', '')))

        flight_data.append(a_e_time)
        flight_data.append(a_a_time)

        # 預計/實際抵達閘門時間
        flight_data.append(None)
        flight_data.append(None)

    # 降落時區
    flight_data.append(a_a_tz)

    # 紀錄該航班網址，若有需要可再重新訪問
    flight_data.append(url)

    return flight_data

### 主程式

In [ ]:
# 航空公司清單
flight_corp = ["EVA", "CAL", "SJX", "TTW"]
day = dtm.get_2days_ago()
columns = get_col()

for corp in flight_corp:
    # 設定航班資訊的檔案路徑
    table_folder = r"C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\Data\old"
    table_file = f"{day}_{corp}_FlightsTable.csv"
    df_table, table_path = pdm.read_or_build(folder=table_folder, file=table_file, columns=columns)

    # 設定航班列表的資料表路徑，並將列表資料讀入
    list_folder = r"C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\Data\old"
    list_file = f"{day}_{corp}_FlightList.csv"

    exist, list_path = pdm.exist_or_not(folder=list_folder, file=list_file)

    if exist:
        df_list = pd.read_csv(list_path)
    else:
        print(f"目前尚無{corp}資料可供查詢")
        continue

    # 設定篩選條件，保留符合條件的（未同步，且日期超過兩天以上）
    # df_list['query_date'] = pd.to_datetime(df_list['query_date'])
    # source = source_list_mask(df_list, df_table)
    source = df_list.copy()

    # 建立dataframe需要的data list
    data = []

    # 建立selenium連線
    # selenium_url = "http://localhost:4444/wd/hub"

    chrome_driver_path = r"C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\tool\chromedriver.exe"
    service = Service(executable_path=chrome_driver_path)

    options = Options()
    options.add_argument("--headless")
    print('建立連線')

    # 根據df_list中的link欄位跑回圈，逐一進入網頁取得html編碼
    for url in source['link']:
        with webdriver.Chrome(service=service, options=options) as driver:
            try:
                page_source = get_page_source(url, driver)

            except Exception as e:
                print(f"無法存取 {url}: {e}")
                continue

        soup = BeautifulSoup(page_source, 'html.parser')
        flight_no = soup('div', class_='flightPageIdent')[0].h1.text.strip()

        # 根據取得的soup物件，開始抓取各項資訊
        print(f'開始查詢{flight_no}班機資訊資訊...')

        try:
            gate = gate_exist(soup)
            print(f'{flight_no}航班有無閘門資訊：{gate}')
            flight_data = crawl_flight_data(
                soup, url, gate_exist=gate)

        except Exception as e:
            print(f'發生錯誤：{e}')

        data.append(flight_data)
        print(f'完成存取{flight_no}航班資料')
        time.sleep(4)

    columns = get_col()
    df_flight = pd.DataFrame(columns=columns, data=data)

    # 將本次爬取航班中，航行未完成（沒有降落時間）的資料先去除，待下次再爬取
    df_flight = df_flight.dropna(subset=['landing_time_actual'])

    # 將爬取的新資料與原本的table資料合併
    df_combine = pd.concat([df_table, df_flight], ignore_index=True)

    # 根據link欄位再去除可能的重複值
    df_combine = df_combine.drop_duplicates(subset='link', keep='first')

    # 將去除重複後的資料存檔
    df_combine.to_csv(table_path, index=False)

    # # 將已經爬取過的航班sync欄位改為1，避免下次重複爬取
    # mask_done = df_list['link'].isin(df_flight['link'])
    # df_list.loc[mask_done, 'sync'] = 1

    # # 將修改後的df_list再存檔回FlightList.csv檔案
    # list_path = os.path.join(list_folder, list_file)
    # df_list.to_csv(list_path, index=False)

print('已更新所有資料！')

建立連線
開始查詢EVA10班機資訊資訊...
EVA10航班有無閘門資訊：1
完成存取EVA10航班資料
開始查詢EVA108班機資訊資訊...
EVA108航班有無閘門資訊：1
完成存取EVA108航班資料
開始查詢EVA132班機資訊資訊...
EVA132航班有無閘門資訊：1
完成存取EVA132航班資料
開始查詢EVA16班機資訊資訊...
EVA16航班有無閘門資訊：1
完成存取EVA16航班資料
開始查詢EVA166班機資訊資訊...
EVA166航班有無閘門資訊：1
完成存取EVA166航班資料
開始查詢EVA170班機資訊資訊...
EVA170航班有無閘門資訊：1
完成存取EVA170航班資料
開始查詢EVA182班機資訊資訊...
EVA182航班有無閘門資訊：1
完成存取EVA182航班資料
開始查詢EVA184班機資訊資訊...
EVA184航班有無閘門資訊：1
完成存取EVA184航班資料
開始查詢EVA192班機資訊資訊...
EVA192航班有無閘門資訊：1
完成存取EVA192航班資料
開始查詢EVA198班機資訊資訊...
EVA198航班有無閘門資訊：1
完成存取EVA198航班資料
無法存取 https://www.flightaware.com/live/flight/id/EVA211-1761351950-schedule-1490p%3a0: HTTPConnectionPool(host='localhost', port=10050): Read timed out. (read timeout=120)
無法存取 https://www.flightaware.com/live/flight/id/EVA225-1761349204-schedule-1045p%3a0: HTTPConnectionPool(host='localhost', port=10050): Read timed out. (read timeout=120)
開始查詢EVA233班機資訊資訊...
EVA233航班有無閘門資訊：1
完成存取EVA233航班資料
開始查詢EVA237班機資訊資訊...
EVA237航班有無閘門資訊：1
完成存取EVA237航班資料
開始查詢EVA257班機資訊資訊...
EVA257航班有無閘門資訊：

Service process refused to terminate gracefully with SIGTERM, escalating to SIGKILL.
Traceback (most recent call last):
  File "c:\Users\add41\AppData\Local\pypoetry\Cache\virtualenvs\flights-data-crawler-HkKwTBFH-py3.11\Lib\site-packages\selenium\webdriver\common\service.py", line 181, in _terminate_process
    self.process.wait(60)
  File "C:\Users\add41\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\add41\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1593, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.TimeoutExpired: Command '['C:\\Users\\add41\\Documents\\Data_Engineer\\Project\\Flights-Data-Crawler\\tool\\chromedriver.exe', '--enable-chrome-logs', '--port=10050']' timed out after 60 seconds


開始查詢EVA265班機資訊資訊...
EVA265航班有無閘門資訊：1
完成存取EVA265航班資料
開始查詢EVA28班機資訊資訊...
EVA28航班有無閘門資訊：1
完成存取EVA28航班資料
開始查詢EVA281班機資訊資訊...
EVA281航班有無閘門資訊：1
完成存取EVA281航班資料
開始查詢EVA32班機資訊資訊...
EVA32航班有無閘門資訊：1
完成存取EVA32航班資料
開始查詢EVA395班機資訊資訊...
EVA395航班有無閘門資訊：1
完成存取EVA395航班資料
開始查詢EVA5班機資訊資訊...
EVA5航班有無閘門資訊：1
完成存取EVA5航班資料
開始查詢EVA52班機資訊資訊...
EVA52航班有無閘門資訊：1
完成存取EVA52航班資料
無法存取 https://www.flightaware.com/live/flight/id/EVA6061-1761328780-schedule-977p%3a0: Message: 

開始查詢EVA62班機資訊資訊...
EVA62航班有無閘門資訊：1
完成存取EVA62航班資料
開始查詢EVA620班機資訊資訊...
EVA620航班有無閘門資訊：0
完成存取EVA620航班資料
開始查詢EVA621班機資訊資訊...
EVA621航班有無閘門資訊：0
完成存取EVA621航班資料
開始查詢EVA637班機資訊資訊...
EVA637航班有無閘門資訊：0
完成存取EVA637航班資料
開始查詢EVA65班機資訊資訊...
EVA65航班有無閘門資訊：1
完成存取EVA65航班資料
開始查詢EVA651班機資訊資訊...
EVA651航班有無閘門資訊：0
完成存取EVA651航班資料
開始查詢EVA658班機資訊資訊...
EVA658航班有無閘門資訊：1
完成存取EVA658航班資料
開始查詢EVA668班機資訊資訊...
EVA668航班有無閘門資訊：1
完成存取EVA668航班資料
開始查詢EVA67班機資訊資訊...
EVA67航班有無閘門資訊：1
完成存取EVA67航班資料
開始查詢EVA68班機資訊資訊...
EVA68航班有無閘門資訊：1
完成存取EVA68航班資料
開始查詢EVA7班機資訊資訊...
EVA7航班有無閘門資訊：1
完成存取EVA7航班資料
